In [51]:
from dotenv import load_dotenv
import os
load_dotenv()

SCHEMA_REGISTRY = os.getenv("SCHEMA_REGISTRY")
BOOTSTRAP_SERVER = os.getenv("BOOTSTRAP_SERVER")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

In [11]:
%%markdown
Producer with schema registry

Producer with schema registry


In [54]:
import uuid
import random

from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer

schema = """{
"type": "record",
"name": "User",
"namespace": "com.upstash",
"fields": [
    {"name": "name", "type": "string"},
    {"name": "favorite_number", "type": "long"}
]
}
"""

schema_registry_client = SchemaRegistryClient({
    'url': SCHEMA_REGISTRY,
    'basic.auth.user.info': USERNAME + ":" + PASSWORD
})

avro_serializer = AvroSerializer(schema_registry_client, schema)

string_serializer = StringSerializer('utf_8')

producer = Producer({'bootstrap.servers':BOOTSTRAP_SERVER,
    'sasl.mechanism':'SCRAM-SHA-256',
    'security.protocol':'SASL_SSL',
    'sasl.username':USERNAME,
    'sasl.password':PASSWORD
})

user = dict(name="user", favorite_number=random.randint(0, 100))
producer.produce(topic="default-topic",
                key=string_serializer(str(uuid.uuid4())),
                value=avro_serializer(user, SerializationContext("default-topic", MessageField.VALUE)))

producer.flush()
print("Message sent successfully")

Message sent successfully


Producer

In [53]:
from kafka import KafkaProducer

producer = KafkaProducer(
    bootstrap_servers=BOOTSTRAP_SERVER,
    sasl_mechanism='SCRAM-SHA-256',
    security_protocol='SASL_SSL',
    sasl_plain_username=USERNAME,
    sasl_plain_password=PASSWORD
)

try:
    producer.send('default-topic', b'Hello from python')
    producer.flush()
    print("Message produced without Avro schema!")
except Exception as e:
    print(f"Error producing message: {e}")
finally:
    producer.close()

Message produced without Avro schema!
